In [ ]:
import psycopg2
import requests
import os

SOURCE_DB_NAME	= os.environ["SOURCE_DB_NAME"]
SOURCE_PASSWORD	= os.environ["SOURCE_PASSWORD"]
SOURCE_USERNAME	= os.environ["SOURCE_USERNAME"]
SOURCE_POINT = os.environ["SOURCE_POINT"]

csv_url = "https://data.bs.ch/api/explore/v2.1/catalog/datasets/100013/exports/csv?lang=de&timezone=Europe%2FZurich&use_labels=true&delimiter=%3B"

try:
    response = requests.get(csv_url)
    response.raise_for_status()

# Download historic data
    with open("100013.csv", "wb") as f:
        f.write(response.content)

    print("Download successful")

except requests.RequestException as e:
    print("Error downloading the CSV file")
    print(e)

    
# connect to database     
try: 
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(SOURCE_POINT, SOURCE_DB_NAME, SOURCE_USERNAME, SOURCE_PASSWORD))
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

# copy traffic table    
conn.set_session(autocommit=True)
try:
    with open('100013.csv', 'r') as f:
        cur.copy_expert("""
            COPY traffic (
            zst_nr,
            sitecode,
            sitename,
            datetimefrom, 
            datetimeto, 
            directionname, 
            lanecode, 
            lanename, 
            valuesapproved, 
            valuesedited, 
            traffictype, 
            total,
            year,
            month,
            day,
            weekday, 
            hourfrom,
            date,
            timefrom,
            timeto,
            dayofyear, 
            zst_id,
            geo_point_2d
            ) FROM STDIN WITH CSV HEADER DELIMITER ';' QUOTE '\"'
        """, f)    
except psycopg2.Error as e:
    print("Error: Copying data")
    print(e)

In [ ]:
try:
# Create latitude and longitude from geo_point_2d
    cur.execute("""
        UPDATE traffic
        SET 
            latitude = ST_X(geo_point_2d::geometry),
            longitude = ST_Y(geo_point_2d::geometry)
        """)

except psycopg2.Error as e:
    print("Error:", str(e))

In [ ]:
# split and aggregate traffic table
try:
    cur.execute("""
        INSERT INTO traffic_agg_historic (
            zst_id,
            datetimefrom,
            datetimeto,
            valuesapproved,
            valuesedited,
            traffictype,
            total_sum
            )
        SELECT
            zst_id,
            datetimefrom,
            datetimeto,
            valuesapproved,
            valuesedited,
            traffictype,
            SUM(total) AS total_sum
        FROM 
            traffic
        GROUP BY
            zst_id, datetimefrom, datetimeto, valuesapproved, valuesedited, traffictype;
        """)
except psycopg2.Error as e:
    print("Error: inserting traffic_agg_historic")
    print(e)

In [ ]:
# split datetime table
try:
    cur.execute("""
        INSERT INTO datetime_info_historic (
            datetimefrom,
            datetimeto,
            year,
            month,
            day,
            weekday,
            hourfrom,
            date,
            timefrom,
            timeto,
            dayofyear
        )
        SELECT DISTINCT
            datetimefrom,
            datetimeto,
            year,
            month,
            day,
            weekday,
            hourfrom,
            TO_DATE(date, 'DD.MM.YYYY'),
            timefrom,
            timeto,
            dayofyear
            FROM
                traffic;
            """)
except psycopg2.Error as e:
    print("Error: inserting datetime_info_historic")
    print(e)

In [ ]:
# split location table
try:
    cur.execute("""
        INSERT INTO location_info_historic (
            zst_id,
            sitename,
            latitude,
            longitude
            )
        SELECT DISTINCT
            zst_id,
            sitename,
            latitude,
            longitude
        FROM
            traffic;
        """)
except psycopg2.Error as e:
    print("Error: inserting location_info_historic")
    print(e)

# Delete duplicate locations    
try:
    cur.execute("""
        DELETE FROM location_info_historic t1
            WHERE EXISTS (
                SELECT 1
                FROM location_info_historic t2
                WHERE t1.zst_id = t2.zst_id
                    AND t1.sitename > t2.sitename
            );
        """)
except psycopg2.Error as e:
    print("Error: inserting location_info_historic")
    print(e)

In [ ]:
# close connection
cur.close()
conn.close()